# Часть 1

1. Извлечем zip архив

In [62]:
import zipfile

def extract_encrypted_zip(zip_file_path, password, destination_path):
    try:
        with zipfile.ZipFile(zip_file_path) as zip_file:
            zip_file.setpassword(bytes(password, 'utf-8'))
            zip_file.extractall(path=destination_path)
            print("Файлы успешно извлечены")
    except:
        print(f"Error: Произошла ошибка во время извлечения файла")

In [64]:
zip_file_path = 'protected_archive.zip'
file_name = 'invoice-42369643.html'
destination_path = 'extracted_files/'
password = 'netology'
extract_encrypted_zip(zip_file_path, password, destination_path)

Файлы успешно извлечены


2. Добавим api_key от VirusTotal

In [65]:
api_key = input()
api_key

'83584605a43de9456e4c5d0cb61d8ce45855636e77a0b17dab27747731d6275c'

3. Отправим файл на анализ в VirusTotal

In [68]:
import requests

def send_to_virus_total(file_path):
    api_url = "https://www.virustotal.com/api/v3/files"
    headers = {"x-apikey" : api_key}
    with open(file_path, "rb") as file:
        files = {"file": (file_path, file)}
        response = requests.post(api_url, headers=headers, files=files)
        return response.json()

In [69]:
file_path = 'extracted_files/invoice-42369643.html'
vt_response = send_to_virus_total(file_path)
vt_response

{'data': {'type': 'analysis',
  'id': 'NjEyYjU3ZTJhZmU3MDY1ZWJlOTIxMzM3MTcwZGY3ZDQ6MTc0MjEyOTk2NA==',
  'links': {'self': 'https://www.virustotal.com/api/v3/analyses/NjEyYjU3ZTJhZmU3MDY1ZWJlOTIxMzM3MTcwZGY3ZDQ6MTc0MjEyOTk2NA=='}}}

4. Посмотрим на детали анализа файла

In [71]:
analysis_id = vt_response['data']['id']
analysis_id

'NjEyYjU3ZTJhZmU3MDY1ZWJlOTIxMzM3MTcwZGY3ZDQ6MTc0MjEyOTk2NA=='

In [72]:
def get_analysis_info(analysis_id):
    headers = {"x-apikey" : api_key}
    url_analysis_details = f'https://www.virustotal.com/api/v3/analyses/{analysis_id}'
    response_analysis = requests.get(url_analysis_details, headers=headers)
    if response_analysis.status_code == 200:
        analysis_details = response_analysis.json()
        return analysis_details
    else:
        print('Ошибка при получении деталей анализа:', response_analysis.text)
        return ""

In [73]:
analysis_info = get_analysis_info(analysis_id)
analysis_info

{'data': {'id': 'NjEyYjU3ZTJhZmU3MDY1ZWJlOTIxMzM3MTcwZGY3ZDQ6MTc0MjEyOTk2NA==',
  'type': 'analysis',
  'links': {'self': 'https://www.virustotal.com/api/v3/analyses/NjEyYjU3ZTJhZmU3MDY1ZWJlOTIxMzM3MTcwZGY3ZDQ6MTc0MjEyOTk2NA==',
   'item': 'https://www.virustotal.com/api/v3/files/f4dc2e1abf2de55b455c7dfc0bbe66a9d12f7e857cd12f8408dadd7129946ae3'},
  'attributes': {'date': 1742129964,
   'status': 'completed',
   'stats': {'malicious': 27,
    'suspicious': 0,
    'undetected': 35,
    'harmless': 0,
    'timeout': 0,
    'confirmed-timeout': 0,
    'failure': 0,
    'type-unsupported': 15},
   'results': {'Bkav': {'method': 'blacklist',
     'engine_name': 'Bkav',
     'engine_version': '2.0.0.1',
     'engine_update': '20250316',
     'category': 'undetected',
     'result': None},
    'Lionic': {'method': 'blacklist',
     'engine_name': 'Lionic',
     'engine_version': '8.16',
     'engine_update': '20250316',
     'category': 'malicious',
     'result': 'Trojan.Script.Obfuse.4!c'},


In [78]:
results = analysis_info['data']['attributes']['results']
results

{'Bkav': {'method': 'blacklist',
  'engine_name': 'Bkav',
  'engine_version': '2.0.0.1',
  'engine_update': '20250316',
  'category': 'undetected',
  'result': None},
 'Lionic': {'method': 'blacklist',
  'engine_name': 'Lionic',
  'engine_version': '8.16',
  'engine_update': '20250316',
  'category': 'malicious',
  'result': 'Trojan.Script.Obfuse.4!c'},
 'MicroWorld-eScan': {'method': 'blacklist',
  'engine_name': 'MicroWorld-eScan',
  'engine_version': '14.0.409.0',
  'engine_update': '20250316',
  'category': 'malicious',
  'result': 'Trojan.Generic.37575014'},
 'ClamAV': {'method': 'blacklist',
  'engine_name': 'ClamAV',
  'engine_version': '1.4.2.0',
  'engine_update': '20250316',
  'category': 'malicious',
  'result': 'Xls.Dropper.Agent-7398287-0'},
 'CTX': {'method': 'blacklist',
  'engine_name': 'CTX',
  'engine_version': '2024.8.29.1',
  'engine_update': '20250316',
  'category': 'malicious',
  'result': 'txt.trojan.obfuse'},
 'CAT-QuickHeal': {'method': 'blacklist',
  'engine_

In [88]:
result_dict = [values for values in results.values() 
                    if 'category' in values and values['category'] != 'undetected' 
                    and values['category'] != 'type-unsupported']

len(result_dict)

27

Выведем список антивирусов, которые обнаружили угрозы, и которые не обнаружили

In [90]:
malicious_detected = [values['engine_name'] for values in results.values() 
                    if 'category' in values and values['category'] == 'malicious']
malicious_not_detected = [values['engine_name'] for values in results.values() 
                    if 'category' in values and values['category'] != 'malicious']
print(f'Detected: {malicious_detected}')
print(f'Not detected: {malicious_not_detected}')

Detected: ['Lionic', 'MicroWorld-eScan', 'ClamAV', 'CTX', 'Sangfor', 'Symantec', 'ESET-NOD32', 'Avast', 'Kaspersky', 'BitDefender', 'Tencent', 'Emsisoft', 'DrWeb', 'VIPRE', 'Ikarus', 'FireEye', 'Google', 'Fortinet', 'Kingsoft', 'Arcabit', 'ViRobot', 'Microsoft', 'Varist', 'ALYac', 'GData', 'AVG', 'alibabacloud']
Not detected: ['Bkav', 'CAT-QuickHeal', 'Skyhigh', 'McAfee', 'Malwarebytes', 'Zillya', 'K7AntiVirus', 'K7GW', 'CrowdStrike', 'Baidu', 'VirIT', 'TrendMicro-HouseCall', 'Cynet', 'NANO-Antivirus', 'SUPERAntiSpyware', 'F-Secure', 'TrendMicro', 'CMC', 'Sophos', 'Jiangmin', 'Avira', 'Antiy-AVL', 'Gridinsoft', 'Xcitium', 'ZoneAlarm', 'AhnLab-V3', 'Acronis', 'VBA32', 'TACHYON', 'Zoner', 'Rising', 'Yandex', 'huorong', 'MaxSecure', 'Panda', 'McAfeeD', 'Avast-Mobile', 'SymantecMobileInsight', 'BitDefenderFalx', 'Elastic', 'DeepInstinct', 'Webroot', 'APEX', 'Paloalto', 'Alibaba', 'Trapmine', 'Cylance', 'SentinelOne', 'tehtris', 'Trustlook']


5. Получим ответ о проведении анализа файла

In [74]:
import time

def get_file_report(file_id):
    headers = { "x-apikey": api_key }
    if file_id:
        url_behaviour = f'https://www.virustotal.com/api/v3/files/{file_id}/behaviour_summary'
        # Ждём некоторое время, чтобы дать VirusTotal время на анализ файла
        print('Ожидаем анализа файла...')
        time.sleep(30)
        response_behaviour = requests.get(url_behaviour, headers=headers)

        if response_behaviour.status_code == 200:
            print('Отчет о поведении получен:')
            behaviour_data = response_behaviour.json()
            return behaviour_data
        else:
            print('Ошибка при получении отчета о поведении:', response_behaviour.text)
    else:
        print('ID файла не найден, пропускаем получение отчета о поведении')
        return {}

In [75]:
file_id = analysis_info["meta"]["file_info"]['sha256']
file_report = get_file_report(file_id)
file_report

Ожидаем анализа файла...
Отчет о поведении получен:


{'data': {'processes_created': ['C:\\Windows\\System32\\WindowsPowerShell\\v1.0\\powershell.exe" -noLogo -ExecutionPolicy unrestricted -file "C:\\Users\\user\\Desktop\\script.ps1',
   'C:\\Windows\\System32\\conhost.exe C:\\Windows\\system32\\conhost.exe 0xffffffff -ForceV1'],
  'registry_keys_opened': ['HKEY_CURRENT_USER\\Software\\Microsoft\\Command Processor\\DisableUNCCheck',
   'HKEY_CLASSES_ROOT\\Drive\\shellex\\FolderExtensions\\{fbeb8a05-beee-4442-804e-409d6c4515e9}',
   'HKEY_LOCAL_MACHINE\\Software\\Microsoft\\Windows\\CurrentVersion\\Explorer\\FolderDescriptions',
   'HKEY_CURRENT_USER\\Software\\Policies\\Microsoft\\Windows\\System',
   'HKEY_CLASSES_ROOT\\SystemFileAssociations\\.',
   'HKEY_CURRENT_USER\\Software\\Microsoft\\Command Processor\\DelayedExpansion',
   'HKEY_LOCAL_MACHINE\\SOFTWARE\\Wow6432Node\\Microsoft\\Command Processor\\CompletionChar',
   'HKEY_CURRENT_USER\\Software\\Microsoft\\Windows\\CurrentVersion\\Explorer\\Advanced\\NoNetCrawling',
   'HKEY_LOCAL

In [1]:
import zipfile

def list_zip_files(zip_file_path):
    try:
        with zipfile.ZipFile(zip_file_path, 'r') as zip_file:
            file_list = zip_file.namelist()
            print("Files in the ZIP archive:")
            for file in file_list:
                print(file)
    except zipfile.BadZipFile as e:
        print(f"Error: {e}")



In [2]:
# Example usage
list_zip_files('protected_archive.zip')

Files in the ZIP archive:
invoice-42369643.html


In [6]:
zip_file_path = 'protected_archive.zip'
file_name = 'invoice-42369643.html'  # Use the full path if it's nested in the ZIP
destination_path = 'extracted_files/'
password = 'netology'
extract_encrypted_zip(zip_file_path, password, destination_path)


Files extracted successfully.


In [7]:
pip install vt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 243.8 kB/s eta 0:00:00 kB/s eta 0:00:01:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 1.1 MB/s eta 0:00:000:00:0136m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 2.5 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.3/145.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 743.1 kB/s eta 0:00:001m6.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.4/128.4 kB 1.4 MB/s eta 0:00:00 MB/s eta 0:00:01
  Created wheel for vt: filename=vt-3.1.3.7-py3-none-any.whl size=39445 sha256=b47ca056b

In [12]:
pip uninstall --yes vt

Found existing installation: vt 3.1.3.7
Uninstalling vt-3.1.3.7:
  Successfully uninstalled vt-3.1.3.7
Note: you may need to restart the kernel to use updated packages.


In [29]:
api_key = '83584605a43de9456e4c5d0cb61d8ce45855636e77a0b17dab27747731d6275c'
file_path = 'extracted_files/invoice-42369643.html'

In [23]:
import os

def get_file_size(file_path):
    try:
        file_size = os.path.getsize(file_path)
        return file_size
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
    except OSError:
        print(f"OS error occurred for file '{file_path}'.")

# Example usage
size = get_file_size(file_path)
if size is not None:
    print(f"File size is {size} bytes")


File size is 48950 bytes


In [17]:
import requests

In [ ]:
import requests

def getResponse():
    api_url = "https://www.virustotal.com/api/v3/files"
    headers = {"x-apikey" : api_key}
    with open(file_path, "rb") as file:
        files = {"file": (file_path, file)}
        response = requests.post(api_url, headers=headers, files=files)
        return response

In [33]:
myResponse = getResponse()
print(myResponse.json())

{'data': {'type': 'analysis', 'id': 'NjEyYjU3ZTJhZmU3MDY1ZWJlOTIxMzM3MTcwZGY3ZDQ6MTc0MjEyNzYwNg==', 'links': {'self': 'https://www.virustotal.com/api/v3/analyses/NjEyYjU3ZTJhZmU3MDY1ZWJlOTIxMzM3MTcwZGY3ZDQ6MTc0MjEyNzYwNg=='}}}


In [41]:
print(myResponse.json()['data'])
analysis_id = myResponse.json()['data']['id']
print(analysis_id)

{'type': 'analysis', 'id': 'NjEyYjU3ZTJhZmU3MDY1ZWJlOTIxMzM3MTcwZGY3ZDQ6MTc0MjEyNzYwNg==', 'links': {'self': 'https://www.virustotal.com/api/v3/analyses/NjEyYjU3ZTJhZmU3MDY1ZWJlOTIxMzM3MTcwZGY3ZDQ6MTc0MjEyNzYwNg=='}}
NjEyYjU3ZTJhZmU3MDY1ZWJlOTIxMzM3MTcwZGY3ZDQ6MTc0MjEyNzYwNg==


In [ ]:
def get_analysis_info():
    headers = {"x-apikey" : api_key}
    url_analysis_details = f'https://www.virustotal.com/api/v3/analyses/{analysis_id}'
    response_analysis = requests.get(url_analysis_details, headers=headers)
    if response_analysis.status_code == 200:
        analysis_details = response_analysis.json()
        return analysis_details
        # Извлечение SHA-256 хеша файла из деталей анализа для получения file_id
        # file_id = analysis_details["data"]["meta"]["file_info"]["sha256"]
        # print('SHA-256 ID файла:', file_id)
    else:
        print('Ошибка при получении деталей анализа:', response_analysis.text)
        return ""

In [ ]:
analysisInfo = get_analysis_info()


AttributeError: 'dict' object has no attribute 'json'

In [47]:
print(analysisInfo)

{'data': {'id': 'NjEyYjU3ZTJhZmU3MDY1ZWJlOTIxMzM3MTcwZGY3ZDQ6MTc0MjEyNzYwNg==', 'type': 'analysis', 'links': {'self': 'https://www.virustotal.com/api/v3/analyses/NjEyYjU3ZTJhZmU3MDY1ZWJlOTIxMzM3MTcwZGY3ZDQ6MTc0MjEyNzYwNg==', 'item': 'https://www.virustotal.com/api/v3/files/f4dc2e1abf2de55b455c7dfc0bbe66a9d12f7e857cd12f8408dadd7129946ae3'}, 'attributes': {'stats': {'malicious': 27, 'suspicious': 0, 'undetected': 35, 'harmless': 0, 'timeout': 0, 'confirmed-timeout': 0, 'failure': 0, 'type-unsupported': 15}, 'results': {'Bkav': {'method': 'blacklist', 'engine_name': 'Bkav', 'engine_version': '2.0.0.1', 'engine_update': '20250316', 'category': 'undetected', 'result': None}, 'Lionic': {'method': 'blacklist', 'engine_name': 'Lionic', 'engine_version': '8.16', 'engine_update': '20250316', 'category': 'malicious', 'result': 'Trojan.Script.Obfuse.4!c'}, 'MicroWorld-eScan': {'method': 'blacklist', 'engine_name': 'MicroWorld-eScan', 'engine_version': '14.0.409.0', 'engine_update': '20250316', 'ca

In [52]:
import json
print(json.dumps(analysisInfo, indent=4))

{
    "data": {
        "id": "NjEyYjU3ZTJhZmU3MDY1ZWJlOTIxMzM3MTcwZGY3ZDQ6MTc0MjEyNzYwNg==",
        "type": "analysis",
        "links": {
            "self": "https://www.virustotal.com/api/v3/analyses/NjEyYjU3ZTJhZmU3MDY1ZWJlOTIxMzM3MTcwZGY3ZDQ6MTc0MjEyNzYwNg==",
            "item": "https://www.virustotal.com/api/v3/files/f4dc2e1abf2de55b455c7dfc0bbe66a9d12f7e857cd12f8408dadd7129946ae3"
        },
        "attributes": {
            "stats": {
                "malicious": 27,
                "suspicious": 0,
                "undetected": 35,
                "harmless": 0,
                "timeout": 0,
                "confirmed-timeout": 0,
                "failure": 0,
                "type-unsupported": 15
            },
            "results": {
                "Bkav": {
                    "method": "blacklist",
                    "engine_name": "Bkav",
                    "engine_version": "2.0.0.1",
                    "engine_update": "20250316",
                    "catego

In [54]:
print(analysisInfo["meta"]["file_info"]['sha256'])

f4dc2e1abf2de55b455c7dfc0bbe66a9d12f7e857cd12f8408dadd7129946ae3


In [55]:
file_id = analysisInfo["meta"]["file_info"]['sha256']

In [56]:
import time
headers = {
    "x-apikey": api_key,
    "Accept": "application/json",
}
if file_id:
    url_behaviour = f'https://www.virustotal.com/api/v3/files/{file_id}/behaviour_summary'
    # Ждём некоторое время, чтобы дать VirusTotal время на анализ файла
    print('Ожидаем анализа файла...')
    # time.sleep(30) # Подождите хотя бы 30 секунд перед проверкой. Может потребоваться больше времени
    response_behaviour = requests.get(url_behaviour, headers=headers)

    if response_behaviour.status_code == 200:
        print('Отчет о поведении получен:')
        behaviour_data = response_behaviour.json()
        print(behaviour_data)
    else:
        print('Ошибка при получении отчета о поведении:', response_behaviour.text)
else:
    print('ID файла не найден, пропускаем получение отчета о поведении')

Ожидаем анализа файла...
Отчет о поведении получен:
{'data': {'modules_loaded': ['KERNEL32.DLL', 'SHELL32.DLL', 'OLE32.DLL', 'CRYPTBASE.DLL', 'PROPSYS.DLL', 'COMCTL32.DLL', 'OLEAUT32.DLL', 'ADVAPI32.DLL', 'NTMARTA.DLL', 'SECHOST.DLL', 'PROFAPI.DLL', 'APPHELP.DLL', 'SETUPAPI.DLL', 'ADVAPI32.dll', 'api-ms-win-core-synch-l1-2-0', 'api-ms-win-core-fibers-l1-1-1', 'kernel32', 'api-ms-win-core-localization-l1-2-1', 'C:\\Windows\\Microsoft.NET\\Framework64\\v4.0.30319\\mscoreei.dll', 'SHLWAPI.dll', 'api-ms-win-appmodel-runtime-l1-1-2.dll', 'api-ms-win-appmodel-runtime-l1-1-0.dll', 'VERSION.dll', 'C:\\Windows\\Microsoft.NET\\Framework64\\v4.0.30319\\clr.dll', 'USER32.dll', 'api-ms-win-core-quirks-l1-1-0.dll', 'C:\\Windows\\Microsoft.NET\\Framework64\\v4.0.30319\\mscoree.dll', 'mscoree.dll', 'ntdll', 'C:\\Windows\\system32\\combase.dll', 'C:\\Windows\\assembly\\NativeImages_v4.0.30319_64\\mscorlib\\3597805b7d7dce423abb491985dd28e8\\mscorlib.ni.dll', 'C:\\Windows\\Microsoft.NET\\Framework64\\v4.

In [57]:
print(json.dumps(behaviour_data, indent=4))

{
    "data": {
        "modules_loaded": [
            "KERNEL32.DLL",
            "SHELL32.DLL",
            "OLE32.DLL",
            "CRYPTBASE.DLL",
            "PROPSYS.DLL",
            "COMCTL32.DLL",
            "OLEAUT32.DLL",
            "ADVAPI32.DLL",
            "NTMARTA.DLL",
            "SECHOST.DLL",
            "PROFAPI.DLL",
            "APPHELP.DLL",
            "SETUPAPI.DLL",
            "ADVAPI32.dll",
            "api-ms-win-core-synch-l1-2-0",
            "api-ms-win-core-fibers-l1-1-1",
            "kernel32",
            "api-ms-win-core-localization-l1-2-1",
            "C:\\Windows\\Microsoft.NET\\Framework64\\v4.0.30319\\mscoreei.dll",
            "SHLWAPI.dll",
            "api-ms-win-appmodel-runtime-l1-1-2.dll",
            "api-ms-win-appmodel-runtime-l1-1-0.dll",
            "VERSION.dll",
            "C:\\Windows\\Microsoft.NET\\Framework64\\v4.0.30319\\clr.dll",
            "USER32.dll",
            "api-ms-win-core-quirks-l1-1-0.dll",
            "C

In [21]:
# Загрузка файла
url_upload = 'https://www.virustotal.com/api/v3/files'
headers = {
    "x-apikey": api_key,
    "Accept": "application/json",
}
files = {'file': open(file_path, 'rb')}
response_upload = requests.post(url_upload, headers=headers, files=files)
if response_upload.status_code == 200:
    print('Файл успешно загружен')
    print(response_upload.json)
    upload_data = response_upload.json()
    file_id = upload_data['data']['id']
    print('ID файла:', file_id)
else:
    print('Ошибка при загрузке файла:', response_upload.text)
    file_id = None

Файл успешно загружен
<bound method Response.json of <Response [200]>>
ID файла: NjEyYjU3ZTJhZmU3MDY1ZWJlOTIxMzM3MTcwZGY3ZDQ6MTc0MjEyNDg5NQ==


In [28]:
import time
headers = {
    "x-apikey": api_key,
    "Accept": "application/json",
}
if file_id:
    url_behaviour = f'https://www.virustotal.com/api/v3/files/{file_id}/behaviour_summary'
    # Ждём некоторое время, чтобы дать VirusTotal время на анализ файла
    print('Ожидаем анализа файла...')
    # time.sleep(30) # Подождите хотя бы 30 секунд перед проверкой. Может потребоваться больше времени
    response_behaviour = requests.get(url_behaviour, headers=headers)

    if response_behaviour.status_code == 200:
        print('Отчет о поведении получен:')
        behaviour_data = response_behaviour.json()
        print(behaviour_data)
    else:
        print('Ошибка при получении отчета о поведении:', response_behaviour.text)
else:
    print('ID файла не найден, пропускаем получение отчета о поведении')

Ожидаем анализа файла...
Ошибка при получении отчета о поведении: {"error":{"code":"NotFoundError","message":"Resource not found."}}



_U891iur2802Kdu43@asd

# Часть 2

In [91]:
json_data = '''
[
    {
        "Program": "LibreOffice",
        "Version": "6.0.7"
    },
    {
        "Program": "7zip",
        "Version": "18.05"
    },
    {
        "Program": "Adobe Reader",
        "Version": "2018.011.20035"
    },
    {
        "Program": "nginx",
        "Version": "1.14.0"
    },
    {
        "Program": "Apache HTTP Server",
        "Version": "2.4.29"
    },
    {
        "Program": "DjVu Reader",
        "Version": "2.0.0.27"
    },
    {
        "Program": "Wireshark",
        "Version": "2.6.1"
    },
    {
        "Program": "Notepad++",
        "Version": "7.5.6"
    },
    {
        "Program": "Google Chrome",
        "Version": "68.0.3440.106"
    },
    {
        "Program": "Mozilla Firefox",
        "Version": "61.0.1"
    }
]
'''

In [92]:
# Parse JSON into a Python dictionary
data = json.loads(json_data)


проверим api key

In [93]:
vulners_api_key = '8K10E4SH7RJ9WMXNO6CHV47VLSEIGT3D1AMMPA5S9Z22Z6TXLDETUF9SUFD4XOH9'

In [94]:
import requests

url = 'https://vulners.com/api/v3/search/lucene/'
data = {
    "query": "Cisco",
    "apiKey": vulners_api_key  # Replace <Your-API-Key-Here> with actual API key
}

response = requests.post(url, json=data)
assert response.status_code == 200, f"Unexpected status code: {response.status_code}"


In [95]:
pip install vulners

Note: you may need to restart the kernel to use updated packages.
